This notebook is used to generate maps in html format based on results files, and cleaned data files. The reason behind creating this was as google cloud would not allow rewriting the html file (in an easy way), and to save the processing time when the application is used. At the time when this is created, a total of 140 maps are needed. This solution cannot be scaled for project upgrades. 

# Imports

In [1]:
import folium
import pandas as pd

# Function to create a map

In [2]:
# Generates a html map with two series of markers inlcuding existing business, and suggested results. 
# Each series require an specific format of dictionary data 
def create_map(center, zoom, result_locations, existing_locations):
    result_map = folium.Map(location=center, zoom_start=13)
    for point in result_locations :
        location = [point['lat'], point['lon']]
        popup = "PCF = {0}, PS ={1}".format(point['PCF_norm'], point['PS_norm'],)
        folium.Marker(location=location,popup=popup,icon=folium.Icon(color='green')).add_to(result_map)
    for point in existing_locations :
        location = [point['latitude'], point['longitude']]
        popup = "{0}".format(point['Name'])
        folium.Marker(location=location,popup=popup,icon=folium.Icon(color='red')).add_to(result_map)
    return (result_map)

# loop over businesses and result ranges to create maps, using the function

In [12]:
data_folder = '/home/payam/workdir/data/'
city = 'Kingston'
businesses = ['bakery','dairy','fitness center','grocery', 'hair care','laundry', 'Swimming pool']
for business in businesses : 
    data_filename = data_folder + business + '_in_' + city + '_cleaned.csv'
    results_filename = data_folder  + business + '_in_' + city + '_results.csv'
    similar_businesses_df = pd.read_csv(data_filename)
    results_df = pd.read_csv(results_filename)
    sim_points = similar_businesses_df.to_dict('records')
    for pareto_start in range(0,100,5) :
        selected_df = results_df.sort_values('PS', ascending =True)[pareto_start:pareto_start+5]
        target_points = selected_df.to_dict('records')
        result_map =  create_map([44.24, -76.58], 13, target_points, sim_points)
        map_name = '/home/payam/workdir/maps/map_'+ business + '_in_' + city + '_' + str(pareto_start) + '_' + str(pareto_start+5) + '.html'
        result_map.save(map_name)